# E-Commerce Basic Analytics 
This notebook demonstrates how to perform basic analysis of revenue metrics and customer activities from e-commerce transactions, product sales, and activity tracking.

In [1]:
# loading necessary libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [2]:
# creating spark session
spark = SparkSession.builder.appName("E-Commerce Logs Analysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/22 15:35:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# loading datasets
products_df = spark.read.format("csv").option("header", "true").load("/Users/cherifamanatoulhasy/Downloads/Pyspark-E-Commerce-Logs-Analysis/src/data_generation/data/raw/products.csv")
logs_df = spark.read.format("csv").option("header", "true").load("/Users/cherifamanatoulhasy/Downloads/Pyspark-E-Commerce-Logs-Analysis/src/data_generation/data/raw/user_logs.csv")
transactions_df = spark.read.format("csv").option("header", "true").load("/Users/cherifamanatoulhasy/Downloads/Pyspark-E-Commerce-Logs-Analysis/src/data_generation/data/raw/transactions.csv")

In [5]:
products_df.toPandas().head()

,product_id,product_name,category,price,brand,stock_quantity,rating
0,product_0000,Sharable bifurcated algorithm,Electronics,744.12,Adidas,228,1.6
1,product_0001,User-centric even-keeled encryption,Clothing,594.58,Nike,30,1.4
2,product_0002,Face-to-face client-driven support,Electronics,565.62,Under Armour,225,2.8
3,product_0003,Customer-focused systematic support,Electronics,761.21,Penguin,714,2.7
4,product_0004,Quality-focused background parallelism,Home & Garden,223.15,Hachette,104,1.4


In [6]:
logs_df.toPandas().head()

,user_id,timestamp,page_url,session_id,action,device_type,duration_seconds
0,user_000265,2025-10-22 23:31:16.433097,/cart,session_cb763127,click,Mobile,329
1,user_000652,2025-10-22 23:34:50.433097,/category/beauty,session_b6f37bd4,click,Mobile,169
2,user_000082,2025-10-22 23:38:39.433097,/cart,session_7aa4f052,click,Mobile,295
3,user_000179,2025-10-22 23:40:16.433097,/,session_80e2811b,remove_from_cart,Mobile,129
4,user_000024,2025-10-22 23:44:47.433097,/returns,session_ad0205f5,view,Mobile,78


In [7]:
transactions_df.toPandas().head()

,transaction_id,user_id,product_id,quantity,unit_price,amount,timestamp,payment_method,status
0,cf40268c-cb3a-4d96-98d1-aef1a2610257,user_000445,product_0094,1,226.54,226.54,2025-10-22 23:30:40.562860,paypal,completed
1,f487f4ce-e20e-4a24-a74e-65bff19f48e6,user_000445,product_0003,1,588.43,588.43,2025-10-22 23:30:40.562860,paypal,cancelled
2,86142f38-c195-442c-ad8a-ad7e08f16514,user_000218,product_0053,1,879.67,879.67,2025-10-22 23:30:40.562860,gift_card,pending
3,796a9818-9ade-44de-ac6d-ca5257d050cf,user_000218,product_0007,2,344.76,689.52,2025-10-22 23:30:40.562860,debit_card,completed
4,7c7519b7-5fbc-4473-a136-a1f40435aa87,user_000499,product_0047,3,185.1,555.3,2025-10-22 23:32:40.562860,paypal,completed


In [7]:
#Basic Analytics
# 1. Total number of visitors
def count_total_visitors(logs_file):
    """This function counts the total number of unique visitors in the logs file.
    Args:
        logs_file (DataFrame): The logs DataFrame containing user activity logs.
    Returns:
        int: The total number of unique visitors.
    """
    total_visitors = logs_file.select("user_id").distinct().count()
    return total_visitors

total_visitors = count_total_visitors(logs_file=logs_df)
print(f"Total number of visitors: {total_visitors}")

Total number of visitors: 1000


In [8]:
# The most visited pages
def most_visited_pages(logs_file):
    """This function returns the most visited pages from the logs DataFrame.
    Args:
        logs_df (DataFrame): The logs DataFrame containing user activity logs.
    Returns:
        DataFrame: A DataFrame containing the most visited pages and their visit counts.
    """
    page_visits = logs_file.groupBy("page_url").count().orderBy(desc("count"))
    return page_visits
page_visits = most_visited_pages(logs_file=logs_df)
page_visits.show(10)

+----------------+-----+
|        page_url|count|
+----------------+-----+
| /category/books|  536|
|/category/sports|  533|
|        /account|  522|
|  /category/home|  520|
|          /about|  517|
|         /search|  515|
|       /checkout|  515|
|               /|  510|
|         /offers|  505|
|  /category/toys|  501|
+----------------+-----+
only showing top 10 rows


In [10]:

# The total revenue
def calculate_total_sales_revenue(transactions_file):
    """This function calculates the total sales revenue from the transactions DataFrame.
    Args:
        transactions_file (DataFrame): The transactions DataFrame containing transaction records.
    Returns:
        float: The total revenue.
    """
    total_sales_revenue = transactions_df.agg(sum("amount").alias("T.S.R")).collect()[0]["T.S.R"]
    return total_sales_revenue
total_sales_revenue = calculate_total_sales_revenue(transactions_file=transactions_df)
print(f"Total Sales Revenue: {total_sales_revenue} EUR")

Total Sales Revenue: 6713085.289999983 EUR


In [13]:
# the most 10 produits sales
def top_selling_products(transactions_file):
    """This function returns the top selling products from the transactions DataFrame.
    Args:
        transactions_file (DataFrame): The transactions DataFrame containing transaction records. 
    Returns:
        DataFrame: A DataFrame containing the top selling products and their sales counts.
    """  
    top_products = transactions_df.groupBy("product_id").count().orderBy(desc("count"))
    return top_products

top_products = top_selling_products(transactions_file=transactions_df)
top_products.show(10)


+------------+-----+
|  product_id|count|
+------------+-----+
|product_0112|   70|
|product_0062|   68|
|product_0135|   59|
|product_0014|   59|
|product_0137|   59|
|product_0154|   59|
|product_0099|   58|
|product_0039|   58|
|product_0118|   57|
|product_0018|   57|
+------------+-----+
only showing top 10 rows


In [14]:
# function to calculate the rate of conversion
def calculate_conversion_rate(logs_file, transactions_file):

    """Calculate the conversion rate as the percentage of visitors who made a purchase.
    Args:
        logs_file (DataFrame): The logs DataFrame containing user activity logs.
        transactions_file (DataFrame): The transactions DataFrame containing transaction records.
    Returns:
        float: The conversion rate as a percentage.
    """
    total_visitors = logs_file.select("user_id").distinct().count()
    purchasing_visitors = transactions_file.select("user_id").distinct().count()
    conversion_rate = (purchasing_visitors / total_visitors) * 100 if total_visitors > 0 else 0
    return conversion_rate

# Calculate and print the conversion rate
conversion_rate = calculate_conversion_rate(logs_file=logs_df, transactions_file = transactions_df)
print(f"Conversion Rate: {conversion_rate:.2f}%")


Conversion Rate: 30.00%


In [15]:
spark.stop()